# 02_Filtering

In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
import pandas as pd
import numpy as np
import os
import tqdm
import pysam

In [82]:
colon_dna_fn = '/broad/thechenlab/pipeline_slidetag/data/240609_SL-EXC_0308_A22KHFYLT3/count'
colon_rna_fn = '/broad/thechenlab/Benno/experiments/xBO153/cellranger'
colon_spatial_fn = '/broad/thechenlab/Andy/slide_tag_dna/colon_droplet/data/Mehdi_smaht_153_celltypes_locations.csv'
# bam_fn = '/broad/thechenlab/Benno/experiments/xBO153/cellranger/xBO153_GEX_240625/outs/possorted_genome_bam.bam' # RNA
bam_fn = '/broad/thechenlab/Benno/experiments/xBO153/bams/xBO153_G.markdup.bam' 
maf_fn = '/broad/thechenlab/ClaudiaC/droplet_DNA/data/xSDS_variants.txt'

# mapping barcodes
atac_bcs_fn = '/broad/thechenlab/Andy/software/cellranger-arc-2.0.2/lib/python/atac/barcodes/737K-arc-v1.txt.gz'
rna_bcs_fn = '/broad/thechenlab/Andy/software/cellranger-arc-2.0.2/lib/python/cellranger/barcodes/737K-arc-v1.txt.gz'

cytobands_fn = '/broad/thechenlab/ClaudiaC/droplet_DNA/refs/cytoBand.txt.gz'
convert_cb_fn = '/broad/thechenlab/ClaudiaC/droplet_DNA/refs/convert_cb.tsv'

Filter cells by:
- Has a spatial barcode
- RNA quality?

Filter mutations after filtering cells:
- Loci detected in X% of high quality cells
- At least 1 or 2 cells with alt count

In [88]:
convert_cb_df = pd.read_csv(convert_cb_fn, sep='\t')
colon_spatial_df = pd.read_csv(colon_spatial_fn, sep=',', index_col=0)

In [107]:
cb_df = convert_cb_df.set_index('CB').copy()
subset_idx = (cb_df['RNA'] + '-1').isin(colon_spatial_df.index)
cb_df.loc[subset_idx, colon_spatial_df.columns] = colon_spatial_df.loc[(cb_df.loc[subset_idx, 'RNA'] + '-1').tolist()]
cb_df['has_spatial'] = cb_df['x'].notna()
cb_df['has_celltype_annot'] = cb_df['cell_type'].notna()

In [108]:
cb_df

,ATAC,RNA,cell_type,x,y,cell_type_broad,has_spatial,has_celltype_annot
CB,,,,,,,,
GTAACACACCCGCTGT,ACAGCGGGTGTGTTAC,AAACAGCCAAACAACA,NaN,NaN,NaN,NaN,False,False
AAGAACAACCCGCTGT,ACAGCGGGTTGTTCTT,AAACAGCCAAACATAG,NaN,NaN,NaN,NaN,False,False
GCCTGTTACCCGCTGT,ACAGCGGGTAACAGGC,AAACAGCCAAACCCTA,NaN,NaN,NaN,NaN,False,False
TTCGCGCACCCGCTGT,ACAGCGGGTGCGCGAA,AAACAGCCAAACCTAT,NaN,NaN,NaN,NaN,False,False
ATGGAGGACCCGCTGT,ACAGCGGGTCCTCCAT,AAACAGCCAAACCTTG,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...
TGCTATGTGAAGTACG,CGTACTTCACATAGCA,TTTGTTGGTTTGGGTA,NaN,NaN,NaN,NaN,False,False
TTGCATGTGAAGTACG,CGTACTTCACATGCAA,TTTGTTGGTTTGGTTC,NaN,NaN,NaN,NaN,False,False
TACAACCTGAAGTACG,CGTACTTCAGGTTGTA,TTTGTTGGTTTGTCTA,NaN,NaN,NaN,NaN,False,False
